In [ ]:
#@title # 1. Configuración del Entorno e Instalación de Dependencias
# Se instalan las versiones más recientes de las librerías necesarias.
# TensorFlow Recommenders para la etapa de recuperación.
# Ya no se necesita tensorflow-ranking.
!pip install -q --upgrade tensorflow-recommenders

# Importar las librerías principales
import os
import pprint
import tempfile
from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Imprimir versiones para asegurar compatibilidad
print(f"TensorFlow Version: {tf.__version__}")
print(f"TFRS Version: {tfrs.__version__}")


#@title # 2. Simulación de Datos (Modelo de Datos Proporcionado)
# --- Parámetros de la Simulación ---
NUM_CLIENTES = 2000
NUM_ESTABLECIMIENTOS = 1000
NUM_TRANSACCIONES = 150000

# --- Tabla: Clientes ---
print("Generando tabla de Clientes...")
cliente_ids = [f"cliente_{i}" for i in range(NUM_CLIENTES)]
edades = np.random.randint(18, 70, size=NUM_CLIENTES)
generos = np.random.choice(['M', 'F'], NUM_CLIENTES, p=[0.5, 0.5])
segmentos = np.random.choice(['Premium', 'Clasico', 'Nuevo'], NUM_CLIENTES, p=[0.15, 0.65, 0.2])
clientes_df = pd.DataFrame({
    'id_cliente': cliente_ids,
    'edad': edades,
    'genero': generos,
    'segmento_cliente': segmentos
})

# --- Tabla: Establecimientos ---
print("Generando tabla de Establecimientos...")
establecimiento_ids = [f"est_{i}" for i in range(NUM_ESTABLECIMIENTOS)]
rubros = ['Restaurante', 'Grifo', 'Retail', 'Entretenimiento', 'Salud', 'Servicios']
sub_rubros_map = {
    'Restaurante': ['Comida Rapida', 'Gourmet', 'Cafeteria', 'Polleria'],
    'Grifo': ['Gasohol 95', 'Gasohol 90', 'Diesel', 'Tienda Conveniencia'],
    'Retail': ['Supermercado', 'Tienda por Departamento', 'Farmacia', 'Ferreteria'],
    'Entretenimiento': ['Cine', 'Casino', 'Teatro', 'Bowling'],
    'Salud': ['Clinica', 'Laboratorio', 'Optica'],
    'Servicios': ['Peluqueria', 'Lavanderia', 'Gimnasio']
}
distritos_lima = ['Miraflores', 'San Isidro', 'Surco', 'La Molina', 'San Borja', 'Lince', 'Pueblo Libre']

establecimientos_data = []
for est_id in establecimiento_ids:
    rubro = np.random.choice(rubros, p=[0.3, 0.15, 0.3, 0.1, 0.1, 0.05])
    sub_rubro = np.random.choice(sub_rubros_map[rubro])
    distrito = np.random.choice(distritos_lima)
    establecimientos_data.append([est_id, f"Local {est_id.split('_')[1]}", rubro, sub_rubro, 'Lima', 'Lima', distrito])

establecimientos_df = pd.DataFrame(establecimientos_data, columns=[
    'id_establecimiento', 'nombre_establecimiento', 'rubro', 'sub_rubro',
    'departamento', 'provincia', 'distrito'
])

# --- Tabla: Transacciones ---
print("Generando tabla de Transacciones...")
tx_cliente_ids = np.random.choice(cliente_ids, NUM_TRANSACCIONES)
tx_establecimiento_ids = np.random.choice(establecimiento_ids, NUM_TRANSACCIONES)
montos = np.round(np.random.lognormal(mean=3.5, sigma=1.0, size=NUM_TRANSACCIONES) + 5, 2)
timestamps = np.random.randint(1641013200, 1704085199, size=NUM_TRANSACCIONES) # Transacciones en 2022-2023
cuotas = np.random.choice([0, 1, 3, 6, 12], NUM_TRANSACCIONES, p=[0.6, 0.1, 0.15, 0.1, 0.05])

transacciones_df = pd.DataFrame({
    'id_cliente': tx_cliente_ids,
    'id_establecimiento': tx_establecimiento_ids,
    'monto_compra': montos,
    'fecha_compra': pd.to_datetime(timestamps, unit='s'),
    'numero_cuotas': cuotas
})
transacciones_df = transacciones_df.drop_duplicates(subset=['id_cliente', 'id_establecimiento', 'fecha_compra']).reset_index(drop=True)
transacciones_df['id_transaccion'] = [f"tx_{i}" for i in range(len(transacciones_df))]


# --- Creación del DataFrame Unificado para Entrenamiento ---
print("Unificando tablas para el modelo...")
full_df = transacciones_df.merge(clientes_df, on='id_cliente').merge(establecimientos_df, on='id_establecimiento')

print("\n--- Muestra de la Tabla Clientes ---")
print(clientes_df.head())
print(f"\nTotal de clientes: {len(clientes_df)}")

print("\n--- Muestra de la Tabla Establecimientos ---")
print(establecimientos_df.head())
print(f"\nTotal de establecimientos: {len(establecimientos_df)}")

print("\n--- Muestra de la Tabla Transacciones ---")
print(transacciones_df.head())
print(f"\nTotal de transacciones: {len(transacciones_df)}")

print("\n--- Muestra del DataFrame Unificado para Modelado ---")
print(full_df.head())


#@title # 3. Etapa 1: Recuperación con TensorFlow Recommenders (TFRS)

# --- 3.1 Preparación de Datos para TFRS ---
ratings_ds = tf.data.Dataset.from_tensor_slices(
    dict(transacciones_df[['id_cliente', 'id_establecimiento']])
)
establecimientos_ds = tf.data.Dataset.from_tensor_slices(
    dict(establecimientos_df[['id_establecimiento']])
)

cliente_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
cliente_ids_vocabulary.adapt(ratings_ds.map(lambda x: x["id_cliente"]))

establecimiento_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
establecimiento_ids_vocabulary.adapt(establecimientos_ds.map(lambda x: x["id_establecimiento"]))


# --- 3.2 Construcción del Modelo de Dos Torres ---
embedding_dim = 64

cliente_model = tf.keras.Sequential([
    cliente_ids_vocabulary,
    tf.keras.layers.Embedding(cliente_ids_vocabulary.vocabulary_size(), embedding_dim)
])

establecimiento_model = tf.keras.Sequential([
    establecimiento_ids_vocabulary,
    tf.keras.layers.Embedding(establecimiento_ids_vocabulary.vocabulary_size(), embedding_dim)
])

# --- 3.3 Definición de la Tarea de Recuperación y Entrenamiento ---
retrieval_task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=establecimientos_ds.batch(128).map(establecimiento_model)
    )
)

class TwoTowerRetrievalModel(tfrs.Model):
    def __init__(self, user_model, item_model, task):
        super().__init__()
        self.user_model = user_model
        self.item_model = item_model
        self.task = task

    def compute_loss(self, data: Dict, training=False) -> tf.Tensor:
        user_embeddings = self.user_model(data["id_cliente"])
        positive_item_embeddings = self.item_model(data["id_establecimiento"])
        return self.task(user_embeddings, positive_item_embeddings)

retrieval_model = TwoTowerRetrievalModel(cliente_model, establecimiento_model, retrieval_task)
retrieval_model.compile(optimizer=tf.keras.optimizers.Adgrad(learning_rate=0.1))

cached_train = ratings_ds.shuffle(len(transacciones_df)).batch(8192).cache()

print("\nIniciando entrenamiento del modelo de RECUPERACIÓN...")
retrieval_model.fit(cached_train, epochs=3)
print("Entrenamiento de recuperación completado.")


# --- 3.4 Generación de Candidatos ---
retrieval_index = tfrs.layers.ann.BruteForce(retrieval_model.user_model)
retrieval_index.index_from_dataset(
    tf.data.Dataset.from_tensor_slices(
        np.unique(establecimientos_df['id_establecimiento'])
    ).batch(100).map(lambda x: (x, retrieval_model.item_model(x)))
)

test_cliente_id = "cliente_150"
print(f"\n--- Generando candidatos para el cliente: {test_cliente_id} ---")
_, candidate_establecimientos = retrieval_index(tf.constant([test_cliente_id]))

print(f"Top 10 candidatos recuperados para {test_cliente_id}:")
for est in candidate_establecimientos[0, :10].numpy():
    print(est.decode())


#@title # 4. Etapa 2: Clasificación con Keras Puro (Enfoque Pairwise)

# --- 4.1 Preparación de Datos para el Modelo de Ranking ---
print("\nPreparando datos para el modelo de CLASIFICACIÓN...")

# Crear un mapeo de ID de establecimiento a sus características para búsqueda rápida
establecimiento_features_map = establecimientos_df.set_index('id_establecimiento').to_dict('index')

# Crear un mapeo de ID de cliente a sus características
cliente_features_map = clientes_df.set_index('id_cliente').to_dict('index')

# Agrupar transacciones por cliente
interactions_by_client = full_df.groupby('id_cliente')['id_establecimiento'].apply(list).to_dict()

# Crear el dataset de entrenamiento
# Cada ejemplo consistirá en: (características_cliente, características_establecimiento_positivo, características_establecimiento_negativo)
def generate_training_data():
    all_establecimiento_ids = establecimientos_df['id_establecimiento'].unique()
    for client_id, positive_items in interactions_by_client.items():
        client_features = cliente_features_map[client_id]
        for positive_item_id in positive_items:
            positive_item_features = establecimiento_features_map[positive_item_id]

            # Muestrear un item negativo (uno con el que el cliente NO ha interactuado)
            negative_item_id = np.random.choice(all_establecimiento_ids)
            while negative_item_id in positive_items:
                negative_item_id = np.random.choice(all_establecimiento_ids)
            negative_item_features = establecimiento_features_map[negative_item_id]

            yield {
                # Features de cliente
                "segmento_cliente": client_features['segmento_cliente'],
                "edad": client_features['edad'],
                # Features del item positivo
                "pos_rubro": positive_item_features['rubro'],
                "pos_distrito": positive_item_features['distrito'],
                # Features del item negativo
                "neg_rubro": negative_item_features['rubro'],
                "neg_distrito": negative_item_features['distrito'],
            }

# Crear el tf.data.Dataset
ranking_dataset = tf.data.Dataset.from_generator(
    generate_training_data,
    output_signature={
        "segmento_cliente": tf.TensorSpec(shape=(), dtype=tf.string),
        "edad": tf.TensorSpec(shape=(), dtype=tf.int64),
        "pos_rubro": tf.TensorSpec(shape=(), dtype=tf.string),
        "pos_distrito": tf.TensorSpec(shape=(), dtype=tf.string),
        "neg_rubro": tf.TensorSpec(shape=(), dtype=tf.string),
        "neg_distrito": tf.TensorSpec(shape=(), dtype=tf.string),
    }
).shuffle(10000).batch(2048).cache()


# --- 4.2 Construcción del Modelo de Ranking con Keras ---

# El "scorer" es una sub-red que calcula la puntuación de un par (cliente, establecimiento)
class ScorerModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        self.segmento_vocab = tf.keras.layers.StringLookup(vocabulary=np.unique(clientes_df['segmento_cliente']), mask_token=None)
        self.segmento_embedding = tf.keras.layers.Embedding(self.segmento_vocab.vocabulary_size(), embedding_dimension)
        self.rubro_vocab = tf.keras.layers.StringLookup(vocabulary=np.unique(establecimientos_df['rubro']), mask_token=None)
        self.rubro_embedding = tf.keras.layers.Embedding(self.rubro_vocab.vocabulary_size(), embedding_dimension)
        self.distrito_vocab = tf.keras.layers.StringLookup(vocabulary=np.unique(establecimientos_df['distrito']), mask_token=None)
        self.distrito_embedding = tf.keras.layers.Embedding(self.distrito_vocab.vocabulary_size(), embedding_dimension)

        self.edad_normalization = tf.keras.layers.Normalization(axis=None)
        self.edad_normalization.adapt(clientes_df['edad'].astype(np.float32))

        # Red neuronal densa que calcula la puntuación final
        self.scorer = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(1)
        ])

    def call(self, features):
        segmento_emb = self.segmento_embedding(self.segmento_vocab(features["segmento_cliente"]))
        edad_norm = self.edad_normalization(tf.cast(features["edad"], tf.float32))
        rubro_emb = self.rubro_embedding(self.rubro_vocab(features["rubro"]))
        distrito_emb = self.distrito_embedding(self.distrito_vocab(features["distrito"]))

        # Concatenar todas las características
        all_features = tf.concat([
            segmento_emb,
            tf.expand_dims(edad_norm, -1),
            rubro_emb,
            distrito_emb,
        ], axis=-1)

        score = self.scorer(all_features)
        return score

# El modelo de ranking completo que maneja la lógica pairwise
class PairwiseRankingModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.scorer = ScorerModel()

    def call(self, features):
        # En inferencia, solo necesitamos puntuar un item a la vez
        return self.scorer(features)

    def train_step(self, data):
        with tf.GradientTape() as tape:
            # Calcular score para el item positivo
            positive_score = self.scorer({
                "segmento_cliente": data["segmento_cliente"],
                "edad": data["edad"],
                "rubro": data["pos_rubro"],
                "distrito": data["pos_distrito"],
            })

            # Calcular score para el item negativo
            negative_score = self.scorer({
                "segmento_cliente": data["segmento_cliente"],
                "edad": data["edad"],
                "rubro": data["neg_rubro"],
                "distrito": data["neg_distrito"],
            })

            # Calcular la pérdida pairwise hinge loss
            loss = tf.reduce_mean(tf.maximum(0., 1. - (positive_score - negative_score)))

        # Aplicar gradientes
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        return {"loss": loss}

# --- 4.3 Entrenamiento del Modelo de Ranking ---
ranking_model = PairwiseRankingModel()
ranking_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

print("\nIniciando entrenamiento del modelo de CLASIFICACIÓN (Keras Puro)...")
ranking_model.fit(ranking_dataset, epochs=5)
print("Entrenamiento de clasificación completado.")


#@title # 5. Pipeline de Inferencia Completo: Uniendo Todo
def get_recommendations(cliente_id: str, top_n: int = 10):
    """
    Genera recomendaciones personalizadas para un cliente uniendo las etapas
    de recuperación y clasificación.
    """
    print(f"\n--- Generando recomendaciones para {cliente_id} ---")

    # --- ETAPA 1: RECUPERACIÓN ---
    print("Paso 1: Recuperando 100 candidatos con el modelo TFRS...")
    _, candidate_establecimientos_tensor = retrieval_index(tf.constant([cliente_id]))
    candidate_ids = [p.decode() for p in candidate_establecimientos_tensor[0, :100].numpy()]

    # --- ETAPA 2: PREPARACIÓN DE DATOS PARA RANKING ---
    print("Paso 2: Obteniendo características para el modelo de ranking...")
    cliente_info = clientes_df[clientes_df['id_cliente'] == cliente_id]
    if cliente_info.empty:
        print(f"Error: Cliente {cliente_id} no encontrado.")
        return None

    candidates_df = establecimientos_df[establecimientos_df['id_establecimiento'].isin(candidate_ids)]

    # Crear el diccionario de entrada para el modelo de ranking
    num_candidates = len(candidates_df)
    ranking_input = {
        "segmento_cliente": tf.constant([cliente_info['segmento_cliente'].iloc[0]] * num_candidates),
        "edad": tf.constant([cliente_info['edad'].iloc[0]] * num_candidates, dtype=tf.int64),
        "rubro": tf.constant(candidates_df['rubro'].values),
        "distrito": tf.constant(candidates_df['distrito'].values),
    }

    # --- ETAPA 3: RE-RANKING ---
    print("Paso 3: Puntuando candidatos con el modelo de Keras...")
    scores = ranking_model(ranking_input)

    # --- ETAPA 4: ORDENAMIENTO Y RESULTADO FINAL ---
    print(f"Paso 4: Ordenando y devolviendo los top {top_n} establecimientos.")
    candidates_df['ranking_score'] = scores.numpy()
    final_recommendations = candidates_df.sort_values(by='ranking_score', ascending=False).head(top_n)

    return final_recommendations

# --- Probamos el pipeline completo con un par de clientes de ejemplo ---
final_recs = get_recommendations("cliente_500")
if final_recs is not None:
    print("\n--- RECOMENDACIONES FINALES PARA cliente_500 ---")
    print(final_recs[['id_establecimiento', 'nombre_establecimiento', 'rubro', 'distrito', 'ranking_score']])

final_recs_2 = get_recommendations("cliente_1820")
if final_recs_2 is not None:
    print("\n--- RECOMENDACIONES FINALES PARA cliente_1820 ---")
    print(final_recs_2[['id_establecimiento', 'nombre_establecimiento', 'rubro', 'distrito', 'ranking_score']])